In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('base_agencias_ficticia.csv', delimiter=';')
df.head()

,agencia,ti,to,indice_indisp,vol_atms,vol_transacao,grupo_controle,onda_fechamento,anomes,anomes_fechamento,periodo_fechamento
0,7165,5947,95845,0.062048,13,11519,NAO,3,202508,202508,POS
1,7047,4620,84369,0.054759,15,19406,SIM,NAO SE APLICA,202504,NAO SE APLICA,PRE
2,8551,13491,122382,0.110237,11,3719,SIM,NAO SE APLICA,202508,NAO SE APLICA,POS
3,8458,14652,102152,0.143433,16,14827,SIM,NAO SE APLICA,202506,NAO SE APLICA,PRE
4,4965,14491,102722,0.141070,7,10329,SIM,NAO SE APLICA,202505,NAO SE APLICA,PRE


In [11]:
df.dtypes

agencia                 int64
ti                      int64
to                      int64
indice_indisp         float64
vol_atms                int64
vol_transacao           int64
grupo_controle         object
onda_fechamento        object
anomes                  int64
anomes_fechamento      object
periodo_fechamento     object
dtype: object

In [7]:
nao_grupo_controle = df[df['grupo_controle'] == 'NAO']
grupo_controle = df[df['grupo_controle'] == 'SIM']

nao_grupo_controle_pre = nao_grupo_controle[nao_grupo_controle['periodo_fechamento'] == 'PRE']
grupo_controle_pre = grupo_controle[grupo_controle['periodo_fechamento'] == 'PRE']

In [8]:
nao_grupo_controle_pre.head()

,agencia,ti,to,indice_indisp,vol_atms,vol_transacao,grupo_controle,onda_fechamento,anomes,anomes_fechamento,periodo_fechamento
8,2648,7135,91736,0.077778,16,13710,NAO,2,202504,202507,PRE
11,975,9650,92078,0.104802,5,11411,NAO,3,202506,202509,PRE
27,2825,7100,88226,0.080475,7,11356,NAO,3,202508,202509,PRE
34,4271,17741,77347,0.229369,16,8412,NAO,2,202506,202507,PRE
35,2516,11692,104963,0.111392,5,11325,NAO,2,202505,202507,PRE


In [9]:
grupo_controle_pre.head()

,agencia,ti,to,indice_indisp,vol_atms,vol_transacao,grupo_controle,onda_fechamento,anomes,anomes_fechamento,periodo_fechamento
1,7047,4620,84369,0.054759,15,19406,SIM,NAO SE APLICA,202504,NAO SE APLICA,PRE
3,8458,14652,102152,0.143433,16,14827,SIM,NAO SE APLICA,202506,NAO SE APLICA,PRE
4,4965,14491,102722,0.141070,7,10329,SIM,NAO SE APLICA,202505,NAO SE APLICA,PRE
5,8237,11017,96652,0.113986,12,12056,SIM,NAO SE APLICA,202506,NAO SE APLICA,PRE
7,7448,9272,80644,0.114974,6,2016,SIM,NAO SE APLICA,202506,NAO SE APLICA,PRE


In [21]:
grupo_controle_pre_logica_anomes = grupo_controle_pre
nao_grupo_controle_pre_logica_anomes = nao_grupo_controle_pre

grupo_controle_pre_logica_anomes = (
    grupo_controle_pre.groupby('agencia')['anomes'].apply(lambda x: sorted(set(x))).reset_index()
)

nao_grupo_controle_pre_logica_anomes = (
    nao_grupo_controle_pre.groupby('agencia')['anomes'].apply(lambda x: sorted(set(x))).reset_index()
)

grupo_controle_pre_logica_anomes = grupo_controle_pre_logica_anomes.drop_duplicates(subset=['agencia'])
nao_grupo_controle_pre_logica_anomes = nao_grupo_controle_pre_logica_anomes.drop_duplicates(subset=['agencia'])
grupo_controle_pre_logica_anomes.to_csv('test/logica_anomes_validacao.csv')

In [ ]:
resultados_pre_logica = []

for index_nao, row_nao in nao_grupo_controle_pre_logica_anomes.iterrows():
    agencia_nao = row_nao['agencia']
    anomes_nao = row_nao['anomes']

    grupo_controle_filtrado = grupo_controle_pre_logica_anomes[
        grupo_controle_pre_logica_anomes['anomes'].apply(lambda x: x[0] == anomes_nao[0])
    ]

    if grupo_controle_filtrado.empty:
        continue

    grupo_controle_filtrado = grupo_controle_filtrado.sort_values(by='tamanho_anomes', ascending=False)
    top_5_agencias = grupo_controle_filtrado.head(5)

for index_sim, row_sim in grupo_controle_filtrado.iterrows():
    resultados_pre_logica.append({
        'agencia_ref': agencia_nao,
        'anomes_agencia_ref': anomes_nao,
        'agencia_grp_controle': row_sim['agencia'],
        'anomes_grp_controle': row_sim['anomes']
    })

lista_ag_proximas = pd.Dataframe(resultados_pre_logica)
lista_ag_proximas.head()